In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from fastcore.basics import patch
import uuid

class ChewC:
    def __init___(self):
        self.trait = None
        self.genome = None
        self.population = None

        
class Genome:
    def __init__(self, n_chr, n_loci):
        self.ploidy = 2
        self.n_chr = n_chr
        self.n_loci = n_loci
        self.shape = (self.ploidy, self.n_chr, self.n_loci)
        
class Trait:
    def __init__(self, genome, founder_population, target_mean, target_variance):
        #store attributes
        self.target_mean = target_mean
        self.target_variance = target_variance
        
        #sample initial random effects
        random_effects = np.random.randn(g.n_chr, g.n_loci)
        #calculate the founder_population mean and var given these effects
        founder_scores = np.array([random_effects @ x.haplotype for x in population.individuals])
        founder_mean, founder_var = founder_scores.mean(), founder_scores.var()
        #scale the random effects to match our target variance
#         import pdb;pdb.set_trace()
        scaling_factors = np.sqrt(self.target_variance / founder_var)
        random_effects *= scaling_factors
        self.effects = random_effects
        
        self.intercept = target_mean - founder_mean
        
    def __matmul__(self,other):
        if isinstance(other,Individual):
            return self.effects * other.haplotype + self.intercept
        elif isinstance(other, Population):
            return np.sum(np.array([self @ ind + self.intercept for ind in other.individuals]), axis=(1,2))

        
class Trait:
    def __init__(self, genome, founder_population, target_mean, target_variance):
        #store attributes
        self.target_mean = target_mean
        self.target_variance = target_variance
        
        #sample initial random effects
        initial_effects = np.random.randn(g.n_chr * g.n_loci)
        initial_effects -= initial_effects.mean()
        #calculate the founder_population mean and var given these effects
        pop_scores = np.sum(founder_population.get_haplo(),axis=1).reshape(founder_population.size, g.n_chr*g.n_loci)
        founder_scores = np.sum(initial_effects*pop_scores,axis=1)
        founder_mean, founder_var = founder_scores.mean(), founder_scores.var()
        scaling_factors = np.sqrt(self.target_variance / founder_var)
        initial_effects *= scaling_factors
        scaled_effects = initial_effects # ADDS A FIXED EFFECT SOMEHOW????
        self.effects = scaled_effects
        self.intercept = target_mean - founder_mean


        
class Population:
    def __init__(self, genome, size=None):
        self.genome = genome
        self.ploidy = 2
        if size:
            self.size = size
            self.individuals = self._create_initial_population()
        else:
            pass

    def _create_initial_population(self):
        """Create an initial population of founder individuals."""
        return [Individual(self.genome) for _ in range(self.size)]
    
    def get_haplo(self):
        return np.array([x.haplotype for x in self.individuals])
    
    def get_pheno(self):
        return np.array([x.fitness for x in self.individuals])
    
    def trial(self, h2):
        [x.phenotype(h2) for x in self.individuals]

    def __getitem__(self, index):
        return self.individuals[index]

    def __repr__(self):
        return f'Population of size: {self.size}'
    

class Individual:
    def __init__(self, genome,haplotype=None, mother = None, father = None, descendants = 0, source='founder', chewc=None):
        self.genome = genome  # must be Genome class
        self.haplotype = None
        self.source = source  # 'founder', 'cross', 'self', 'dh'
        self.descendents = []
        self.fitness = 0
        self.mother, self.father = mother, father

        # Logic for setting haplotype,mother,father
        if self.source == 'founder':
            self.haplotype= self._generate_random_haplotype()
        else:
            self.haplotype = haplotype
            
        self.make_id()
            
    def __repr__(self):
        return f'Individual with haplotype shape: {self.haplotype.shape}'
    
    def make_id(self):self.id = uuid.uuid4().hex


    def _generate_random_haplotype(self):
        """Generate a random haplotype for the individual."""
        return np.random.choice([0, 1], size=(self.genome.ploidy, self.genome.n_chr, self.genome.n_loci))
    




        
import numpy as np
import matplotlib.pyplot as plt
from fastcore.basics import patch


# to do move this in another module
@patch
def gamete(self:Individual):
    haplotypes = self.haplotype
    def shuffle_chr(chromosome_pair):
        """
        Perform crossover on a pair of chromosomes.

        Parameters:
        chromosome_pair (list): A list of two equal length lists representing chromosomes.

        Returns:
        list: A new chromosome formed by shuffling the given chromosome pair.
        """
        # Ensure the chromosome pair contains two chromosomes of equal length
        assert len(chromosome_pair) == 2
        assert len(chromosome_pair[0]) == len(chromosome_pair[1])

        # Number of crossover points, sampled from a Poisson distribution with λ=1.3
        n_crossover = np.random.poisson(1.3)

        # Determine crossover locations, sampled without replacement from chromosome length
        chromosome_length = len(chromosome_pair[0])
        crossover_locs = np.sort(np.random.choice(chromosome_length, n_crossover, replace=False))

        # Initialize the new chromosome and set the current chromosome to the first one
        new_chromosome = []
        current_chr = 0

        # Perform crossover by alternating segments between the two chromosomes
        last_loc = 0
        for loc in crossover_locs:
            new_chromosome.extend(chromosome_pair[current_chr][last_loc:loc])
            current_chr = 1 - current_chr  # Switch to the other chromosome
            last_loc = loc

        # Append the remaining segment
        new_chromosome.extend(chromosome_pair[current_chr][last_loc:])
        return np.array(new_chromosome)


    # Initialize an empty array to store the shuffled chromosomes
    shuffled_haplotypes = np.zeros_like(haplotypes)
    ploidy, n_chr, n_loci = chewc.genome.shape
    # Iterate over each chromosome and apply the shuffle_chr function
    for i in range(haplotypes.shape[1]):  # Iterate over the chromosomes
        chromosome_pair = haplotypes[:, i, :]  # Extract the chromosome pair (2, 77)
        shuffled_chromosome = shuffle_chr(chromosome_pair)  # Shuffle the chromosome pair    
        shuffled_haplotypes[:, i, :] = np.array(shuffled_chromosome).reshape(1, n_loci)  # Store the shuffled chromosome
    return shuffled_haplotypes[0,:,:]

@patch
def __mul__(self:Individual, partner):
    if isinstance(partner,Individual):
        source = 'cross'
        progeny_haplo = np.vstack((self.gamete(), partner.gamete()))
        
        progeny = Individual(self.genome, progeny_haplo, self.id, partner.id,source=source, chewc = chewc)
        return progeny

@patch
def phenotype(self:Individual, h2):
    breeding_value = chewc.trait @ self
    genetic_variance = np.var(breeding_value, ddof=1)
    environmental_variance = (1 - h2) / h2 * genetic_variance
    phenotype_value = breeding_value + np.random.normal(0, np.sqrt(environmental_variance))
    self.fitness = np.sum(phenotype_value)

    
@patch
def __matmul__(self:Trait,other):
    if isinstance(other,Individual):
#             print(f' intercept {self.intercept}')
        return self.effects * np.sum(other.haplotype,axis=0).flatten()
    else:
        print('ffff')
        
    
#Define the Simulation Parameters
g = Genome(3, 1000)
population = Population(g, size=999)
trait = Trait(g, population,0,1)

#Plug them into ChewC
chewc = ChewC()
chewc.trait = trait
chewc.population = population
chewc.genome = g

chewc.population.trial(h2=1)